In [252]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import requests

data = {
    'ctl00$FormContentPlaceHolder$Panel$categoryDropDownList': 'a027b6c0-07bb-4301-b9b5-1b38dcdc59b6',
    'ctl00$FormContentPlaceHolder$Panel$certificatesCheckBoxList$5': 'on'
}

headers = {'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Content-Length': '10892',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Host': 'myaccount.rid.org',
    'Origin': 'https://myaccount.rid.org',
    'Pragma': 'no-cache',
    'Referer': 'https://myaccount.rid.org/Public/Search/Member.aspx',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'X-MicrosoftAjax': 'Delta=true',
    'X-Requested-With': 'XMLHttpRequest'}

s = requests.Session()
#r = s.get('https://myaccount.rid.org/Public/Search')
#s.headers = headers
r = s.post('https://myaccount.rid.org/Public/Search/Member.aspx', data=data)

for i in r:
    print(i)

b'<!DOCTYPE html>\r\n<!--[if IE 8 ]><html class="ie ie8" lang="en"> <![endif]-->\r\n<!--[if (gte IE 9)|!(IE)]><!-->\r\n<html lang="en">\r'
b'\n<!--<![endif]-->\r\n<head id="Head1" color="~/App_Themes/Rid/css/color.css"><meta charset="utf-8" /><title>\r\n\tFind an RID Member\r'
b'\n</title><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1" /><meta http-equiv="X-UA-Compatibl'
b'e" content="IE=edge" />\r\n    <!--[if lt IE 9]>\r\n    <script src="./App_Themes/Modern/html5shiv.js"></script>\r\n    <![endif]-->\r\n'
b'<link href="../../App_Themes/Rid/css/base.css" type="text/css" rel="stylesheet" /><link href="../../App_Themes/Rid/css/color%20-'
b'%20Copy.css" type="text/css" rel="stylesheet" /><link href="../../App_Themes/Rid/css/color.css" type="text/css" rel="stylesheet"'
b' /><link href="../../App_Themes/Rid/css/icons.css" type="text/css" rel="stylesheet" /><link href="../../App_Themes/Rid/css/jquer'
b'y.share.css" type="text/css" rel="stylesheet" /><li

Need Gecko Driver: https://github.com/mozilla/geckodriver/releases
Put it somewhere to access. I put mine in `/usr/local/bin/geckodriver`. 

In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

# Set up browser and navigate to page
browser = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')
url = 'https://myaccount.rid.org/Public/Search/Member.aspx'
browser.get(url) # navigate to this page

# Fill out form
category = Select(browser.find_element_by_name('ctl00$FormContentPlaceHolder$Panel$categoryDropDownList'))
category.select_by_value('a027b6c0-07bb-4301-b9b5-1b38dcdc59b6')
state = Select(browser.find_element_by_name('ctl00$FormContentPlaceHolder$Panel$stateDropDownList'))
state.select_by_value('26fdccc5-cf2f-e511-80c5-00155d631510')

# Submit form
browser.find_element_by_name('ctl00$FormContentPlaceHolder$Panel$searchButtonStrip$searchButton').click()

def scrape_page(curr_page_num):
    try:
        # Wait for loader to disappear
        loader = browser.find_element_by_id('FormContentPlaceHolder_Panel_resultsGridFormAjaxGridUpdateProgress')
        isLoading = loader.get_attribute('class')
        print('class: ' + isLoading)
        if isLoading == 'UpdateProgress':
            WebDriverWait(browser, 10).until(EC.invisibility_of_element_located((By.ID, 'FormContentPlaceHolder_Panel_resultsGridFormAjaxGridUpdateProgress')))
        
        # Add explicit wait for the result element, due to AJAX loading
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, "FormContentPlaceHolder_Panel_resultsGrid")))
        
        # Scrape results table and add to array of results tables
        results.append(browser.page_source)
        
        # Paginate to the next page of results
        curr_page_num += 1
        paginate(curr_page_num)
    except Exception as e:
        print(e)

def paginate(next_page_num):
    try:
        print('going to ' + str(next_page_num))
        pager_row = browser.find_element_by_class_name('PagerStyle')
        next_page_link = pager_row.find_element_by_link_text(str(next_page_num))
        next_page_link.click()
        scrape_page(next_page_num)
    except NoSuchElementException or ElementClickInterceptedException:
        try:
            print('expanding paginator')
            pager_row = browser.find_element_by_class_name('PagerStyle')
            expands = pager_row.find_elements_by_link_text('...') # need to find second link only
            
            # If we are not on the first page and there is only one '...', we are on the last page
            global onFirstPage
            if onFirstPage:
                expands[0].click()
                onFirstPage = False
            elif not onFirstPage and len(expands) == 1:
                print('Done paginating')
                return
            else:
                expands[len(expands)-1].click() # Expand the last '...' 

            scrape_page(next_page_num)
        except Exception as e:
            print(e)

# Scrape the first page of results
try:
    results = []
    curr_page = 1
    onFirstPage = True
    scrape_page(curr_page)
finally:
    browser.quit()

class: 
going to 2
class: UpdateProgress
going to 3
class: UpdateProgress
going to 4
class: UpdateProgress
going to 5
class: UpdateProgress
going to 6
expanding paginator
class: UpdateProgress
going to 7
class: UpdateProgress
going to 8
class: UpdateProgress
going to 9
class: UpdateProgress
going to 10
class: UpdateProgress
going to 11
expanding paginator
class: UpdateProgress
going to 12
class: UpdateProgress
going to 13
class: UpdateProgress
going to 14
class: UpdateProgress
going to 15
class: UpdateProgress
going to 16
expanding paginator
class: UpdateProgress
going to 17
class: UpdateProgress
going to 18
class: UpdateProgress
going to 19
class: UpdateProgress
going to 20
class: UpdateProgress
going to 21
expanding paginator
class: UpdateProgress
going to 22
class: UpdateProgress
going to 23
class: UpdateProgress
going to 24
class: UpdateProgress
going to 25
class: UpdateProgress
going to 26
expanding paginator
class: UpdateProgress
going to 27
class: UpdateProgress
going to 28
clas

In [6]:
len(results)

59

In [7]:
from bs4 import BeautifulSoup
import csv

headers = None
rows = []

for result in results:
    soup = BeautifulSoup(result, 'lxml')
    table = soup.find(id='FormContentPlaceHolder_Panel_resultsGrid')
    if headers is None:
        headers = [header.text for header in table.find_all('th')]
    for row in table.find_all('tr', class_='RowStyle'):
        rows.append([val.text for val in row.find_all('td')])

with open('output_file.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(row for row in rows if row)
    
!head output_file.csv --lines=100

==> output_file.csv <==










head: --lines=100: No such file or directory


Once you get your table, get the headers by iterating through all the `th` elements, storing them in a list. Then we create a `rows` list that will contain lists representing the contents of each row. This is populated by finding all `td` elements under `tr` tags and taking the text, encoding it in UTF-8. 

Then you open a CSV, writing the headers first and then writing all of the rows, but using `row for row in rows` if row) to eliminate any blank rows.


For input checkmark I needed to get xpath. In inspector > right click on element in Elements tab > Copy > Copy XPath

Open csv back up and export new csv that is cleaned. Delete rows if the person does not have a city and state or a zipcode. Also delete all columns except name, city, state, zipcode.

In [8]:
import csv
with open('output_file.csv', 'rt') as inp, open('edited_rid.csv', 'wt') as out:
    writer = csv.writer(out)
    for row in csv.reader(inp):
        if (row[1].strip() not in (None, "") and row[2].strip() not in (None, "")) or row[3].strip() not in (None, ""):
            writer.writerow((row[0], row[1], row[2], row[3]))